In [1]:
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
%%capture
!wget https://github.com/xprilion/pure-python-rag-with-in-memory-vector-db/raw/main/inmemorydb.py

--2024-05-03 07:38:42--  https://github.com/xprilion/pure-python-rag-with-in-memory-vector-db/raw/main/inmemorydb.py
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/xprilion/pure-python-rag-with-in-memory-vector-db/main/inmemorydb.py [following]
--2024-05-03 07:38:42--  https://raw.githubusercontent.com/xprilion/pure-python-rag-with-in-memory-vector-db/main/inmemorydb.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1495 (1.5K) [text/plain]
Saving to: ‘inmemorydb.py’

inmemorydb.py       100%[===================>]   1.46K  --.-KB/s    in 0s      

2024-05-03 07:38:42 (22.2 MB/s) - ‘inmemorydb.

In [3]:
from inmemorydb import InMemoryVectorDB

In [4]:
memorydb = InMemoryVectorDB()

## Let's test vectorization

In [5]:
collection = memorydb.get_or_create_collection("test")

In [6]:
documents = [
    "Mohan goes to school",
    "Ramesh is a boy",
    "Anita is working on a project",
    "Kriti has some books"
]

metadatas = [
    "mohan-school",
    "ramesh-boy",
    "anita-working-project",
    "kriti-books"
]

ids = [
    "doc1",
    "doc2",
    "doc3",
    "doc4"
]

In [7]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)
bow_embeddings = X.toarray()

In [8]:
collection.add(
    embeddings=bow_embeddings,
    metadatas=metadatas,
    documents=documents,
    ids=ids,
)

In [26]:
search_query = "Who has books?"

In [27]:
y = vectorizer.transform([search_query])

In [28]:
y_embeddings = y.toarray()

In [29]:
results = collection.query(
    query_embeddings=y_embeddings,
    include=["documents", "metadatas"]
)

In [30]:
results

{'ids': ['doc4'],
 'documents': ['Kriti has some books'],
 'metadatas': ['kriti-books']}

In [31]:
y_embeddings = y.toarray()

In [32]:
results = collection.query(
    query_embeddings=y_embeddings,
    include=["documents", "metadatas"]
)

In [33]:
results

{'ids': ['doc4'],
 'documents': ['Kriti has some books'],
 'metadatas': ['kriti-books']}

## Generation

In [34]:
%%capture
!pip install --upgrade google-cloud-aiplatform

In [35]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel

In [36]:
from google.oauth2 import service_account

In [37]:
credentials = service_account.Credentials.from_service_account_file('gcp-adventure-x-3ef20383c9c8.json')

In [38]:
vertexai.init(project="gcp-adventure-x", location="us-central1", credentials=credentials)

In [39]:
model = GenerativeModel("gemini-1.0-pro")

In [40]:
context = "Context: " + results['documents'][0]

In [41]:
context

'Context: Kriti has some books'

In [42]:
response = model.generate_content(
    context + " \n " + search_query
)
print(response.text)

Kriti has some books.
